In [1]:
!pip install numpy==1.26.4 --force-reinstall

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [2]:
!pip install faiss-cpu==1.7.4 mistralai

In [4]:
!pip uninstall -y numpy
!pip install numpy

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 78.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.


In [10]:
# from mistralai import Mistral
# import requests
# import numpy as np
# import faiss
# import os
# from getpass import getpass

# api_key= getpass("Type your API Key")
# client = Mistral(api_key=api_key)
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from getpass import getpass

api_key = getpass("Type your API Key: ")

client = MistralClient(api_key=api_key)

# Example: Send a chat completion request
messages = [ChatMessage(role="user", content="What is the capital of France?")]
response = client.chat(model="mistral-tiny", messages=messages)

print(response.choices[0].message.content)


Type your API Key: ··········
The capital of France is Paris. Paris is one of the most famous cities in the world, known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. It serves as the political, cultural, and economic hub of France.


In [12]:
import requests
response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

In [13]:
f = open('essay.txt', 'w')
f.write(text)
f.close()

In [14]:
len(text)

75014

In [15]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

In [16]:
len(chunks)

37

In [23]:
# import time

# def get_text_embedding(input):
#     while True:
#         try:
#             embeddings_batch_response = client.embeddings.create(
#                 model="mistral-embed",
#                 inputs=input
#             )
#             return embeddings_batch_response.data[0].embedding
#         except Exception as e:
#             if "rate limit" in str(e).lower():
#                 print("Rate limit hit. Sleeping for 5 seconds...")
#                 time.sleep(5)
#             else:
#                 raise e
def get_text_embedding(text):
    response = client.embeddings(
        model="mistral-embed",  # or the model ID you're allowed to use
        input=[text]
    )
    return response.data[0].embedding

In [24]:
import numpy as np
#text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])


In [25]:
text_embeddings.shape

(37, 1024)

In [26]:
text_embeddings

array([[-0.03979492,  0.07733154,  0.00013709, ..., -0.01274109,
        -0.02101135, -0.00264168],
       [-0.03152466,  0.07226562,  0.02961731, ..., -0.01079559,
        -0.01189423, -0.00821686],
       [-0.05905151,  0.06112671,  0.01206207, ..., -0.0226593 ,
         0.00488663, -0.00665283],
       ...,
       [-0.05477905,  0.06890869,  0.02703857, ..., -0.02456665,
        -0.02526855, -0.02687073],
       [-0.03884888,  0.05587769,  0.04718018, ..., -0.01812744,
         0.00926208, -0.00866699],
       [-0.03048706,  0.05831909,  0.01704407, ..., -0.01620483,
        -0.01800537, -0.04415894]])

In [28]:
import faiss
import os
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [29]:
question = "What were the two main things the author worked on before college?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape

(1, 1024)

In [30]:
question_embeddings

array([[-0.05447388,  0.03479004,  0.0375061 , ..., -0.02787781,
        -0.00327492,  0.0029068 ]])

In [31]:
D, I = index.search(question_embeddings, k=2)
print(I)

[[0 3]]


In [32]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then 

In [33]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [37]:
# def run_mistral(user_message, model="mistral-large-latest"):
#     messages = [{"role": "user", "content": user_message}]
#     while True:
#         try:
#             chat_response = client.chat.complete(
#                 model=model,
#                 messages=messages
#             )
#             return chat_response.choices[0].message.content
#         except Exception as e:
#             if "capacity exceeded" in str(e).lower() or "429" in str(e):
#                 print("Model capacity exceeded or rate limited. Sleeping for 10 seconds...")
#                 time.sleep(10)
#             else:
#                 raise e
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [{"role": "user", "content": user_message}]
    while True:
        try:
            chat_response = client.chat(
                model=model,
                messages=messages
            )
            return chat_response.choices[0].message.content
        except Exception as e:
            if "capacity exceeded" in str(e).lower() or "429" in str(e):
                print("Model capacity exceeded or rate limited. Sleeping for 10 seconds...")
                time.sleep(10)
            else:
                raise e


In [38]:
run_mistral(prompt)

'The two main things the author worked on before college were:\n1. Writing (specifically, short stories)\n2. Programming'

In [39]:
run_mistral(prompt, model="mistral-small")  # or "mistral-medium"

'Based on the provided context, the two main things the author worked on before college were writing and programming. The author mentions writing short stories, although they were not very good, and trying to write programs on an IBM 1401 computer, although they struggled to find something to do with it.'

# LangChain

In [40]:
!pip install langchain langchain-mistralai langchain_community mistralai==0.4.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.24
    Uninstalling langchain-0.3.24:
      Successfully uninstalled langchain-0.3.24


In [42]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

# Load data
loader = TextLoader("essay.txt")
docs = loader.load()
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
# Define the embedding model
embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=api_key)
# Create the vector store
vector = FAISS.from_documents(documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()
# Define LLM
model = ChatMistralAI(mistral_api_key=api_key)
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "What were the two main things the author worked on before college?"})
print(response["answer"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/langchain_mistralai/embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


The two main things the author worked on before college were writing and programming. He wrote short stories and tried programming on an IBM 1401 using Fortran. His stories were awful and he couldn't figure out what to do with the 1401 due to the lack of data stored on punch cards. His programming skills improved with the advent of microcomputers, which allowed him to write simple games, a program to predict how high his model rockets would fly, and a word processor.


# LlamaIndex

In [43]:
!pip install llama-index==0.10.55 llama-index-llms-mistralai==0.1.18 llama-index-embeddings-mistralai mistralai==0.4.2

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
INFO: pip is looking at multiple versions of llama-index-embeddings-mistralai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requir

In [37]:
!pip install mistralai==0.3.0 \
            llama-index==0.10.55 \
            llama-index-llms-mistralai==0.1.18 \
            llama-index-embeddings-mistralai==0.1.18

ERROR: Could not find a version that satisfies the requirement llama-index-embeddings-mistralai==0.1.18 (from versions: 0.0.1, 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.1.5, 0.1.6, 0.2.0, 0.3.0)
ERROR: No matching distribution found for llama-index-embeddings-mistralai==0.1.18


In [44]:
import os
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding

# Load data
reader = SimpleDirectoryReader(input_files=["essay.txt"])
documents = reader.load_data()
# Define LLM and embedding model
Settings.llm = MistralAI(model="mistral-medium", api_key=api_key)
Settings.embed_model = MistralAIEmbedding(model_name='mistral-embed', api_key=api_key)
# Create vector store index
index = VectorStoreIndex.from_documents(documents)
# Create query engine
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query(
    "What were the two main things the author worked on before college?"
)
print(str(response))

The two main things the author worked on before college, outside of school, were writing and programming. They wrote short stories and attempted to write programs on an IBM 1401 in 9th grade.


# Haystack

In [45]:
!pip install mistral-haystack==0.0.1 mistralai==0.4.2

In [64]:
# RAG Pipeline Code Review and Recommendations


from typing import Dict, Any, List
from haystack import Pipeline, component
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.dataclasses import ChatMessage
from haystack.utils.auth import Secret

# Import the correct prompt builder
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.converters import TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.writers import DocumentWriter

# Import Mistral-specific components
from haystack_integrations.components.embedders.mistral import MistralDocumentEmbedder, MistralTextEmbedder
from haystack_integrations.components.generators.mistral import MistralChatGenerator

@component
class PromptToChatMessagesConverter:
    @component.output_types(messages=List[ChatMessage])
    def run(self, prompt: str) -> Dict[str, Any]:
        return {"messages": [ChatMessage.from_user(text=prompt)]}

# API Key Security Warning
# IMPORTANT: Do not hardcode API keys in your script. Use environment variables or secure configuration.
api_key = Secret.from_token("AjpaVyhiEb199RERZjRFPi1xKagAC8wK")

# Create document store
document_store = InMemoryDocumentStore()

# Load and process documents
docs = TextFileToDocument().run(sources=["essay.txt"])
split_docs = DocumentSplitter(split_by="passage", split_length=2).run(documents=docs["documents"])

# Embed documents
embeddings = MistralDocumentEmbedder(api_key=api_key).run(documents=split_docs["documents"])
DocumentWriter(document_store=document_store).run(documents=embeddings["documents"])

# Create components
text_embedder = MistralTextEmbedder(api_key=api_key)
retriever = InMemoryEmbeddingRetriever(document_store=document_store)

# Prompt builder with template and explicit required variables
prompt_builder = PromptBuilder(
    template="""Answer the following question based on the contents of the documents.
Question: {{query}}
Documents:
{% for document in documents %}
{{document.content}}
{% endfor %}""",
    # Explicitly set required variables to resolve the warning
    required_variables=["query", "documents"]
)

# Chat message converter
prompt_to_messages = PromptToChatMessagesConverter()

llm = MistralChatGenerator(
    api_key=api_key,
    model='mistral-small'
)

# Create pipeline
rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", text_embedder)
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("prompt_to_messages", prompt_to_messages)
rag_pipeline.add_component("llm", llm)

# Connect components
rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever.documents", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder.prompt", "prompt_to_messages.prompt")
rag_pipeline.connect("prompt_to_messages.messages", "llm.messages")

# Run the pipeline
question = "What were the two main things the author worked on before college?"
result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"query": question},
        "llm": {"generation_kwargs": {"max_tokens": 225}},
    }
)

# Print the generated response
print(result["llm"]["replies"][0].text)


Calculating embeddings: 3it [00:04,  1.52s/it]


Based on the documents provided, the two main things the author worked on before college were writing and programming. The author mentioned that they wrote short stories, although they admitted that their stories were awful and had hardly any plot. They also worked on spam filters and did some painting. Additionally, the author held dinners for a group of friends every Thursday night, which taught them how to cook for groups.
